# Metaflow with Tempo Example

We will train two models and deploy them with tempo within a Metaflow pipeline. To understand the core example see [here](https://tempo.readthedocs.io/en/latest/examples/multi-model/README.html)

![archtecture](architecture.png)

## MetaFlow Prequisites

Install metaflow

```
pip install metaflow
```

[Install Metaflow with remote AWS support](https://docs.metaflow.org/metaflow-on-aws/metaflow-on-aws).




## Tempo Requirements

For deploy to a remote Kubernetes cluster:

### GKE Cluster

Create a GKE cluster and install Seldon Core on it using [Ansible to install Seldon Core on a Kubernetes cluster](https://github.com/SeldonIO/ansible-k8s-collection).

For GKE we will need to create two files in the flow src folder:

```bash
kubeconfig.yaml
gsa-key.json
```

Follow the steps outlined in [GKE server authentication](https://cloud.google.com/kubernetes-engine/docs/how-to/api-server-authentication#environments-without-gcloud).



In [ ]:
!kubectl create ns production

In [ ]:
!kubectl create -f k8s/tempo-pipeline-rbac.yaml -n production

Create a Secret from the `k8s/s3_secret.yaml.tmpl` file by adding your AWS Key that can read from S3 and saving as `k8s/s3_secret.yaml`

In [ ]:
!kubectl create -f k8s/s3_secret.yaml -n production

## Run Flow locally to deploy to Docker

In [1]:
!python src/irisflow.py \
    --environment=conda \
    run --tempo-on-docker true

Metaflow 2.3.2 executing IrisFlow for user:clive
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
Bootstrapping conda environment...(this could take a few minutes)
Including file src/conda.yaml of size 115B 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/a94f1cff7702ed70807d16917bb282f51a28511e
Including file src/gsa-key.json of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/6a24eaae697b14c2d8cdc2dcee1fce38b23b6ce0
Including file src/kubeconfig.yaml of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/00e21f5423ea21f695831e970aa81a283ac25812
2021-08-07 19:15:22.217 Workflow starting (run-id 27):
2021-08-07 19:15:23.659 [27/start/151 (pid 2044924)] Task is starting.
2021-08-07 19:15:27.443 [27/start/151 (pid 2044924)] Task finished successfully.
2021-08-07 19:15:29.009 [27/train_sklear

2021-08-07 18:16:16.954 [27/tempo/155 (pid 2045276)] Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
2021-08-07 18:16:16.954 [27/tempo/155 (pid 2045276)] Collecting cachetools<5.0,>=2.0.0
2021-08-07 18:16:16.954 [27/tempo/155 (pid 2045276)] Using cached cachetools-4.2.2-py3-none-any.whl (11 kB)
2021-08-07 18:16:16.954 [27/tempo/155 (pid 2045276)] Collecting rsa<5,>=3.1.4
2021-08-07 18:16:16.955 [27/tempo/155 (pid 2045276)] Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
2021-08-07 18:16:16.955 [27/tempo/155 (pid 2045276)] Collecting pyasn1-modules>=0.2.1
2021-08-07 18:16:16.955 [27/tempo/155 (pid 2045276)] Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
2021-08-07 18:16:16.955 [27/tempo/155 (pid 2045276)] Collecting pyasn1<0.5.0,>=0.4.6
2021-08-07 18:16:16.955 [27/tempo/155 (pid 2045276)] Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
2021-08-07 18:16:16.955 [27/tempo/155 (pid 2045276)] Collecting pyparsing>=2.0.2
2021-08-07 18:16:16.955 [27/te

2021-08-07 18:16:30.121 [27/tempo/155 (pid 2045276)] Verifying transaction: ...working... done
2021-08-07 18:16:30.261 [27/tempo/155 (pid 2045276)] Executing transaction: ...working... done
2021-08-07 18:17:08.886 [27/tempo/155 (pid 2045276)] Remove all packages in environment /home/clive/anaconda3/envs/tempo-bfa84d96-4681-4fef-bd43-4911a7b8ff79:
2021-08-07 18:17:08.887 [27/tempo/155 (pid 2045276)] 
2021-08-07 18:17:08.887 [27/tempo/155 (pid 2045276)] Insights Manager not initialised as empty URL provided.
2021-08-07 18:17:18.985 [27/tempo/155 (pid 2045276)] {'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32), 'output1': 'xgboost prediction'}
2021-08-07 19:17:21.262 [27/tempo/155 (pid 2045276)] Task finished successfully.
2021-08-07 19:17:23.169 [27/end/156 (pid 2047464)] Task is starting.
2021-08-07 19:17:26.746 [27/end/156 (pid 2047464)] Task finished successfully.
2021-08-07 19:17:27.207 Done!


Use the saved client from the Flow to make predictions

In [2]:
from metaflow import Flow
run = Flow('IrisFlow').latest_run

In [3]:
client = run.data.client_model

In [4]:
import numpy as np
client.predict(np.array([[1, 2, 3, 4]]))

{'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32),
 'output1': 'xgboost prediction'}

## Run Flow on AWS and Deploy to Kubernetes

In [5]:
!python src/irisflow.py \
    --environment=conda \
    --with batch:image=seldonio/seldon-core-s2i-python37-ubi8:1.10.0-dev \
    run

Metaflow 2.3.2 executing IrisFlow for user:clive
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
Bootstrapping conda environment...(this could take a few minutes)
Including file src/conda.yaml of size 115B 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/a94f1cff7702ed70807d16917bb282f51a28511e
Including file src/gsa-key.json of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/6a24eaae697b14c2d8cdc2dcee1fce38b23b6ce0
Including file src/kubeconfig.yaml of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/00e21f5423ea21f695831e970aa81a283ac25812
2021-08-07 19:25:55.509 Workflow starting (run-id 28):
2021-08-07 19:25:57.115 [28/start/158 (pid 2050712)] Task is starting.
2021-08-07 18:25:58.583 [28/start/158 (pid 2050712)] [dc35448a-1263-4847-b5a3-1f7c8ce72e47] Task is starting (status SUBM

2021-08-07 18:28:25.705 [28/join/161 (pid 2051623)] [624716f9-9250-4627-9e46-d62a71cb1c41] Code package downloaded.
2021-08-07 18:28:25.717 [28/join/161 (pid 2051623)] [624716f9-9250-4627-9e46-d62a71cb1c41] Task is starting.
2021-08-07 18:28:26.084 [28/join/161 (pid 2051623)] [624716f9-9250-4627-9e46-d62a71cb1c41] Bootstrapping environment...
2021-08-07 18:28:42.377 [28/join/161 (pid 2051623)] [624716f9-9250-4627-9e46-d62a71cb1c41] Environment bootstrapped.
2021-08-07 18:28:51.036 [28/join/161 (pid 2051623)] [624716f9-9250-4627-9e46-d62a71cb1c41] Task finished with exit code 0.
2021-08-07 19:28:51.922 [28/join/161 (pid 2051623)] Task finished successfully.
2021-08-07 19:28:53.763 [28/tempo/162 (pid 2051921)] Task is starting.
2021-08-07 18:28:54.749 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51] Task is starting (status SUBMITTED)...
2021-08-07 18:28:58.032 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51] Task is starting (status RUNNABLE)...
202

2021-08-07 18:30:16.095 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51] Collecting h11>=0.8
2021-08-07 18:30:16.095 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51]   Using cached h11-0.12.0-py3-none-any.whl (54 kB)
2021-08-07 18:30:16.095 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51] Collecting asgiref>=3.3.4
2021-08-07 18:30:16.095 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51]   Using cached asgiref-3.4.1-py3-none-any.whl (25 kB)
2021-08-07 18:30:16.096 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51] Installing collected packages: zipp, typing-extensions, pycparser, urllib3, pyasn1, importlib-metadata, idna, charset-normalizer, cffi, starlette, six, rsa, requests, pydantic, pyasn1-modules, oauthlib, multidict, h11, cryptography, click, cachetools, asgiref, yarl, websocket-client, uvicorn, requests-oauthlib, pyyaml, python-dateutil, pyparsing, protobuf, orjson, numpy, grpcio, google-

2021-08-07 18:30:28.193 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51]
2021-08-07 18:30:28.194 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51] Remove all packages in environment /opt/conda/envs/tempo-a1d8e8bb-4b2e-4017-8b60-a5bd0a95a426:
2021-08-07 18:30:28.194 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51]
2021-08-07 18:30:59.653 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51] Insights Manager not initialised as empty URL provided.
2021-08-07 18:31:09.816 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51] {'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32), 'output1': 'xgboost prediction'}
2021-08-07 18:31:14.000 [28/tempo/162 (pid 2051921)] [319ba9da-f3dc-437e-a37d-d8b50004dd51] Task finished with exit code 0.
2021-08-07 19:31:14.909 [28/tempo/162 (pid 2051921)] Task finished successfully.
2021-08-07 19:31:15.927 [28/end/163 (pid 2052740)] Task is starting.
2021-08-

Use the saved client from the Flow to make predictions

In [6]:
from metaflow import Flow
run = Flow('IrisFlow').latest_run

In [7]:
client = run.data.client_model

In [8]:
import numpy as np
client.predict(np.array([[1, 2, 3, 4]]))

{'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32),
 'output1': 'xgboost prediction'}